# Fine-tuning Transformer pré-entraîné pour la reconnaissance d'entités nommées


Dans ce notebook, nous utiliserons un modèle [Transformer](https://arxiv.org/abs/1706.03762) , plus particulièrement le modèle [BERT](https://arxiv.org/abs/1810.04805) pré-entraîné. Notre modèle sera composé du transformer et d'une simple couche linéaire.

## Préparer les données


In [ ]:
# !pip install transformers

In [12]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torchtext import data
from torchtext import datasets
from torch.optim import Adam
from torchtext.data import Field, NestedField, BucketIterator
from torchtext.datasets import SequenceTaggingDataset
from transformers import BertTokenizer, BertModel

import numpy as np

import time
import random
import functools

In [13]:
SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

Ensuite, on importe le tokenizer BERT. Cela définit la manière dont le texte dans le modèle doit être traité, mais contient surtout le vocabulaire avec lequel le modèle BERT a été pré-entraîné. Nous utiliserons le tokenizer et le modèle `bert-base-uncased`. Il a été formé sur du texte en minuscules.

Afin d'utiliser des modèles pré-entraînés pour le NLP, le vocabulaire utilisé doit correspondre exactement à celui du modèle pré-entraîné.

In [14]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

Une autre chose que nous devons faire est de nous assurer que la séquence d'entrée est formatée de la même manière que le modèle BERT a été formé.

BERT a été entraîné sur des séquences commençant par un jeton `[CLS]`.

Donc, la séquence des tokens :

```python
text = ['jack', 'went', 'to', 'the', 'shop']
```

doit devenir :

```python
text = ['[CLS]', 'jack', 'went', 'to', 'the', 'shop']
```

En plus de faire correspondre nos vocabulaires, nous devons également nous assurer que nos jetons padding et unk correspondent à ceux utilisés dans le modèle pré-entraîné. Par défaut, TorchText utilise `<pad>` et `<unk>`, mais le modèle BERT utilise `[PAD]` et `[UNK]`.


In [15]:
init_token = tokenizer.cls_token
pad_token = tokenizer.pad_token
unk_token = tokenizer.unk_token

print(init_token, pad_token, unk_token)

[CLS] [PAD] [UNK]


Nous nous intéressons principalement aux représentations numériques des tokens spéciaux. En effet, nous n'utilisons pas le module de vocabulaire de TorchText, mais celui fourni par le modèle pré-entraîné.

Nous obtenons les index des tokens spéciaux en les passant par la fonction `convert_tokens_to_ids` du tokenizer.

In [16]:
init_token_idx = tokenizer.convert_tokens_to_ids(init_token)
pad_token_idx = tokenizer.convert_tokens_to_ids(pad_token)
unk_token_idx = tokenizer.convert_tokens_to_ids(unk_token)

print(init_token_idx, pad_token_idx, unk_token_idx)

101 0 100


Une autre chose est que le modèle pré-entraîné a été formé sur des séquences d'une longueur maximale et nous devons nous assurer que nos séquences sont également coupées à cette longueur.

In [17]:
max_input_length = tokenizer.max_model_input_sizes['bert-base-uncased']

print(max_input_length)

512


Ensuite, nous définissons deux fonctions qui utilisent notre vocabulaire.

Le premier coupera la séquence de tokens à la longueur maximale souhaitée, spécifiée par notre modèle pré-entraîné, puis convertira les tokens en index en les passant à travers le vocabulaire. C'est ce que nous allons utiliser sur notre séquence d'entrée que nous voulons tagger.

Nous coupons en fait les tokens à `max_input_length-1`, car nous devons ajouter le token spécial` [CLS] `au début de la séquence.

In [18]:
def cut_and_convert_to_id(tokens, tokenizer, max_input_length):
    tokens = tokens[:max_input_length-1]
    tokens = tokenizer.convert_tokens_to_ids(tokens)
    return tokens

La deuxième fonction coupe simplement la séquence à la longueur maximale. Ceci est utilisé pour nos tags. Nous ne transmettons pas les tags à travers le vocabulaire du modèle pré-entraîné, car le vocabulaire n'a été construit que pour les phrases anglaises, et non pour les tags. Nous allons alors construire nous-mêmes le vocabulaire des tags.

In [19]:
def cut_to_max_length(tokens, max_input_length):
    tokens = tokens[:max_input_length-1]
    return tokens

Nous devons passer les deux fonctions ci-dessus au `Field`, l'abstraction TorchText qui gère une grande partie du traitement des données pour nous. Nous utilisons les `functools` de Python qui nous permettent de passer des fonctions qui ont déjà certains de leurs arguments fournis.

In [20]:
text_preprocessor = functools.partial(cut_and_convert_to_id,
                                      tokenizer = tokenizer,
                                      max_input_length = max_input_length)

tag_preprocessor = functools.partial(cut_to_max_length,
                                     max_input_length = max_input_length)

On définit les `Field` et on importe les données en utilisant ces `Field`.

In [27]:
TEXT = data.Field(use_vocab = False,
                  lower = True,
                  preprocessing = text_preprocessor,
                  init_token = init_token_idx,
                  pad_token = pad_token_idx,
                  unk_token = unk_token_idx)

TAG = data.Field(unk_token = None,
                     init_token = '<pad>',
                     preprocessing = tag_preprocessor)


train_data, valid_data, test_data = data.TabularDataset.splits(
        path="data_ner/",
        train="train.csv",
        validation="valid.csv",
        test="test.csv", format='csv', skip_header=True,
        fields=[('text', TEXT), ('tag', TAG)])
    

Nous pouvons vérifier un exemple en l'affichant. Comme nous avons déjà numérisé notre "texte" en utilisant le vocabulaire du modèle pré-entraîné, il s'agit déjà d'une suite d'entiers. Les tags doivent encore être numérisées.

In [29]:
print(vars(train_data.examples[6]))

{'text': [1000, 2057, 2079, 100, 2490, 2151, 2107, 12832, 2138, 2057, 2079, 100, 2156, 2151, 5286, 2005, 2009, 1010, 1000, 1996, 3222, 100, 2708, 14056, 100, 3158, 4315, 14674, 2409, 1037, 2739, 27918, 1012], 'tag': ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-ORG', 'O', 'O', 'O', 'B-PER', 'I-PER', 'I-PER', 'I-PER', 'O', 'O', 'O', 'O', 'O']}


Notre prochaine étape consiste à construire le vocabulaire des tags afin qu'elles puissent être numérisées pendant l'entraînement. Nous faisons cela en utilisant la méthode `.build_vocab` du champ sur` train_data`.

In [30]:
TAG.build_vocab(train_data)

print(TAG.vocab.stoi)

defaultdict(None, {'<pad>': 0, 'O': 1, 'B-LOC': 2, 'B-PER': 3, 'B-ORG': 4, 'I-PER': 5, 'I-ORG': 6, 'B-MISC': 7, 'I-LOC': 8, 'I-MISC': 9})


Ensuite, nous définissons nos itérateurs. Cela définira comment les batchs de données sont fournis lors de l'entraînement. Nous définissons une taille de batch et définissons `device`, qui placera automatiquement notre batch sur le GPU, si nous en avons un.

Le modèle BERT est assez grand, donc la taille du batch ici est généralement plus petite que d'habitude. 

In [48]:
BATCH_SIZE = 32

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE,
    device = device, sort = False)

## Construire le modèle

La prochaine étape consiste à définir notre modèle. Le modèle est relativement simple, avec toutes les pièces compliquées contenues dans le module BERT dont nous n'avons pas à nous soucier. Nous pouvons considérer le BERT comme une couche d'embeddings et tout ce que nous faisons est d'ajouter une couche linéaire au-dessus de ces embeddings pour prédire le tag pour chaque token dans la séquence d'entrée. 

![](https://github.com/bentrevett/pytorch-pos-tagging/blob/master/assets/pos-bert.png?raw=1)

Une chose à noter est que nous ne définissons pas de `embedding_dim` pour notre modèle, c'est la taille de la sortie du modèle BERT prétrainé et nous ne pouvons pas la changer. Ainsi, nous obtenons simplement `embedding_dim` de l'attribut` hidden_size` du modèle.

BERT veut également des séquences avec l'élément batch en premier, c'est pourquoi nous permutons notre séquence d'entrée avant de la passer à BERT.

In [49]:
class BERTNER(nn.Module):
    def __init__(self,
                 bert,
                 output_dim, 
                 dropout):
        
        super().__init__()
        
        self.bert = bert
        
        embedding_dim = bert.config.to_dict()['hidden_size']
    
        self.fc = nn.Linear(embedding_dim, output_dim)       
        self.dropout = nn.Dropout(dropout)
        
            
    def forward(self, text, chars):
  
        #text = [sent len, batch size]
    
        text = text.permute(1, 0)
        #text = [batch size, sent len]  
        embedded = self.dropout(self.bert(text)[0])
        
        #embedded = [batch size, seq len, emb dim]      
        embedded = embedded.permute(1, 0, 2)           
        #embedded = [sent len, batch size, emb dim]
        
        predictions = self.fc(self.dropout(embedded))
        
        #predictions = [sent len, batch size, output dim]
        
        return predictions

Ensuite, nous chargeons le modèle prétrainé BERT non casé - avant nous avons uniquement chargé le tokenizer associé au modèle.

In [50]:
bert = BertModel.from_pretrained('bert-base-uncased')

## Entraînement

In [51]:
OUTPUT_DIM = len(TAG.vocab)
DROPOUT = 0.25

model = BERTNER(bert,
                      OUTPUT_DIM, 
                      DROPOUT)

In [68]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'Le modèle a {count_parameters(model):,} paramètres à entraîner.')

Le modèle a 109,489,930 paramètres à entraîner.


Ensuite, nous définissons notre optimiseur. Habituellement, lors du fine tuning, nous utilisons un learning rate inférieur à la normale, c'est parce que nous ne voulons pas changer radicalement les paramètres car cela peut amener notre modèle à oublier ce qu'il a appris.

On prend 5e-5 (0.00005) car c'est une valeur recommandée dans le papier de BERT.

In [53]:
LEARNING_RATE = 5e-5

optimizer = optim.Adam(model.parameters(), lr = LEARNING_RATE)

Nous définissons une fonction de perte, en veillant à ignorer les pertes chaque fois que le tag cible est un token de remplissage `[PAD]`.

In [54]:
TAG_PAD_IDX = TAG.vocab.stoi[TAG.pad_token]

criterion = nn.CrossEntropyLoss(ignore_index = TAG_PAD_IDX)

Ensuite, nous plaçons le modèle sur le GPU, si nous en avons un.

In [55]:
model = model.to(device)
criterion = criterion.to(device)

On définit une fonction qui calcule le F1 score à chaque batch, en ignorant les tokens `[PAD]` et `O`.

In [67]:
from sklearn.metrics import f1_score
def f1_loss(preds, y, tag_pad_idx):
    index_o = TAG.vocab.stoi["O"]
    positive_labels = [i for i in range(len(TAG.vocab.itos))
                           if i not in (tag_pad_idx, index_o)]
    _, pred = torch.max(preds, 1)
    pred = pred.data.cpu().numpy() 
    tags = y.data.cpu().numpy()
    f1 = f1_score(
            y_true=tags,
            y_pred=pred,
            labels=positive_labels,
            average="micro"
        ) 
       
    return f1

Nous utilisons également la fonction suivante pour calculer l'accuracy par tag.

In [ ]:
def accuracy_per_tag(predictions, tags):
    n_tags = len(TAG.vocab)
    class_correct = list(0 for i in range(n_tags))
    class_total = list(0 for i in range(n_tags))
    acc = list(0 for i in range(n_tags))
    _, pred = torch.max(predictions, 1)
    # # compare predictions to true label
    correct = np.squeeze(pred.eq(tags.data.view_as(pred)))
    # # calculate test accuracy for each object class
    for i in range(BATCH_SIZE):
        label = tags.data[i]
        class_correct[label] += correct[i].item()
        class_total[label] += 1
    for i in range(n_tags):
        if np.sum(class_total[i]) == 0 and np.sum(class_correct[i]) ==0:
            res = 100
        else:
            res = 100 * class_correct[i] / class_total[i]
        acc[i] = res, np.sum(class_correct[i]), np.sum(class_total[i])
        
    return acc  

Nous définissons ensuite nos fonctions `train` et` evaluate` pour entraîner et tester notre modèle. 

In [57]:
def train(model, iterator, optimizer, criterion, tag_pad_idx):
    
    epoch_loss = 0
    epoch_f1 = 0
    
    model.train()
    
    for batch in iterator:
        
        text = batch.text
        tags = batch.tag
                
        optimizer.zero_grad()
        
        #text = [sent len, batch size]
        
        predictions = model(text)
        
        #predictions = [sent len, batch size, output dim]
        #tags = [sent len, batch size]
        
        predictions = predictions.view(-1, predictions.shape[-1])
        tags = tags.view(-1)
        
        #predictions = [sent len * batch size, output dim]
        #tags = [sent len * batch size]
        
        loss = criterion(predictions, tags)
        acc = accuracy_per_tag(predictions, tags)
        f1 = f1_loss(predictions, tags, tag_pad_idx)                
       
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_f1 += f1.item()
        
    return epoch_loss / len(iterator), acc, epoch_f1 / len(iterator)

In [58]:
def evaluate(model, iterator, criterion, tag_pad_idx):
    
    epoch_loss = 0
    epoch_f1 = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            text = batch.text
            tags = batch.tag
            
            predictions = model(text)
            
            predictions = predictions.view(-1, predictions.shape[-1])
            tags = tags.view(-1)
            
            loss = criterion(predictions, tags)
            
            acc = accuracy_per_tag(predictions, tags)
            f1 = f1_loss(predictions, tags, tag_pad_idx) 

            epoch_loss += loss.item()
            epoch_f1 += f1.item()
        
    return epoch_loss / len(iterator), acc, epoch_f1 / len(iterator)

On peut maintenant entraîner notre modèle.

In [60]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

N_EPOCHS = 20

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc, train_f1 = train(model, train_iterator, optimizer, criterion, TAG_PAD_IDX)
    valid_loss, valid_acc, valid_f1 = evaluate(model, valid_iterator, criterion, TAG_PAD_IDX)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut2-model.pt')
    if epoch%5 == 0:
        print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
        print(f'\tTrain Loss: {train_loss:.3f} | Train F1 score: {train_f1*100:.2f}%')
        print(f'\t Val. Loss: {valid_loss:.3f} |  Val. F1 score: {valid_f1*100:.2f}%')

Epoch: 01 | Epoch Time: 4m 33s
	Train Loss: 0.310 | Train Acc: 90.96%
	 Val. Loss: 0.131 |  Val. Acc: 96.42%
Epoch: 02 | Epoch Time: 3m 58s
	Train Loss: 0.102 | Train Acc: 97.03%
	 Val. Loss: 0.093 |  Val. Acc: 97.44%


On affiche les résultats des accuracy par tags :
 - pour le train : 

In [ ]:
n_tags = len(TAG.vocab)
for i in range(n_tags):   
    print('Train Accuracy of %5s: %2d%% (%2d/%2d)' % (
           TAG.vocab.itos[i], train_acc[i][0],
           train_acc[i][1], train_acc[i][2]))  

 - pour les données de validation : 

In [ ]:
for i in range(n_tags):
    print('Valid Accuracy of %5s: %2d%% (%2d/%2d)' % (
           TAG.vocab.itos[i], valid_acc[i][0],
           valid_acc[i][1], valid_acc[i][2]))
  

 - pour le test : 

In [ ]:
model.load_state_dict(torch.load('tut5-model.pt'))

test_loss, test_acc, test_f1 = evaluate(model, test_iterator, criterion, TAG_PAD_IDX)
n_tags = len(TAG.vocab)
for i in range(n_tags):   
    print('Test Accuracy of %5s: %2d%% (%2d/%2d)' % (
           TAG.vocab.itos[i], test_acc[i][0],
           test_acc[i][1], test_acc[i][2]))
print(f'Test Loss: {test_loss:.3f} |  Test F1 score: {test_f1*100:.2f}%')

## Inférence


Nous allons maintenant voir comment utiliser notre modèle pour tagger des phrases réelles.

Si nous transmettons une chaîne de caractères, cela signifie que nous devons la diviser en tokens individuels, ce que nous faisons en utilisant la fonction `tokenize` du` tokenizer`. Ensuite, on numérise les tokens de la même manière que nous l'avons fait auparavant, en utilisant `convert_tokens_to_ids`. Ensuite, nous ajoutons le token index `[CLS]` au début de la séquence.

Nous passons ensuite la séquence de texte à travers notre modèle pour obtenir une prédiction pour chaque token, puis découpons les prédictions pour le token `[CLS]`.

In [62]:
def tag_sentence(model, device, sentence, tokenizer, text_field, tag_field):
    
    model.eval()
    
    if isinstance(sentence, str):
        tokens = tokenizer.tokenize(sentence)
    else:
        tokens = sentence
    
    numericalized_tokens = tokenizer.convert_tokens_to_ids(tokens)
    numericalized_tokens = [text_field.init_token] + numericalized_tokens
        
    unk_idx = text_field.unk_token
    
    unks = [t for t, n in zip(tokens, numericalized_tokens) if n == unk_idx]
    
    token_tensor = torch.LongTensor(numericalized_tokens)
    
    token_tensor = token_tensor.unsqueeze(-1).to(device)
         
    predictions = model(token_tensor)
    
    top_predictions = predictions.argmax(-1)
    
    predicted_tags = [tag_field.vocab.itos[t.item()] for t in top_predictions]
    
    predicted_tags = predicted_tags[1:]
        
    assert len(tokens) == len(predicted_tags)
    
    return tokens, predicted_tags, unks

Nous pouvons ensuite exécuter des exemples dans notre modèle et afficher les tags prédits.

 - Exemple issu de notre jeu de données : 

In [ ]:
example_index = 1

sentence = vars(train_data.examples[example_index])['text']
actual_tags = vars(train_data.examples[example_index])['tag']

print(sentence)
tokens, pred_tags, unks = tag_sentence(model, 
                                       device, 
                                       sentence, 
                                       TEXT, 
                                       TAG,
                                       CHAR)

print("Pred. Tag\tActual Tag\tCorrect?\tToken\n")

for token, pred_tag, actual_tag in zip(tokens, pred_tags, actual_tags):
    correct = '✔' if pred_tag == actual_tag else '✘'
    print(f"{pred_tag}\t\t{actual_tag}\t\t{correct}\t\t{token}")

 - Exemple quelconque :

In [ ]:
sentence = 'The will deliver a speech about the conflict in Sao Paulo at tomorrow in Anne Mary with Jack.'

tokens, tags, unks = tag_sentence(model, 
                                  device, 
                                  sentence, 
                                  TEXT, 
                                  TAG,
                                  CHAR)

print(unks)
print("Pred. Tag\tToken\n")


for token, tag in zip(tokens, tags):
    print(f"{tag}\t\t{token}")

In [66]:
print("Pred. Tag\tToken\n")

for token, tag in zip(tokens, tags):
    print(f"{tag}\t\t{token}")

Pred. Tag	Token

O		the
O		queen
O		will
O		deliver
O		a
O		speech
O		about
O		the
O		conflict
O		in
B-LOC		north
I-LOC		korea
O		at
O		1
O		##pm
O		tomorrow
O		.
